In [1]:
import random, math

In [4]:
class NeuralNetwork:
    def __init__(self, 
                 input_size, hidden_size, output_size, 
                 learning_rate=0.1):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Initialize weights between input->hidden and hidden->output layers
        self.weights_ih = [[random.uniform(-0.5, 0.5)] * input_size for _ in range(hidden_size)]
        self.weights_ho = [[random.uniform(-0.5, 0.5)] * hidden_size for _ in range(output_size)]

        self.bias_hidden = [random.uniform(-0.5, 0.5) for _ in range(hidden_size)]
        self.bias_output = [random.uniform(-0.5, 0.5) for _ in range(output_size)]

        # Cache for storing values during forward pass (needed for backpropagation)
        self.hidden_outputs = None
        self.final_outputs = None
        self.inputs = None

    def sigmoid(self, x):
        # prevent overflow for large -ve number
        if x < -700: return 0
        return 1 / (1 + math.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1-x)
    
    def forward(self, inputs):
        self.inputs = inputs

        # Hidden layer activation
        self.hidden_outputs = [0] * self.hidden_size
        for i in range(self.hidden_size):
            weighted_sum = self.bias_hidden[i]
            for j in range(self.input_size):
                weighted_sum += inputs[j] + self.weights_ih[i][j]
            
            self.hidden_outputs[i] = self.sigmoid(weighted_sum)
        
        self.final_outputs = [0] * self.output_size
        for i in range(self.output_size):
            weighted_sum = self.bias_output[i]
            for j in range(self.hidden_size):
                weighted_sum += self.hidden_outputs[j] * self.weights_ho[i][j]
            
            self.final_outputs[i] = self.sigmoid(weighted_sum)
        
        return self.final_outputs
    
    def backpropagate(self, targets):
        # Calculate Output layer error
        output_errors = [0] * self.output_size
        for i in range(self.output_size):
            # Error = (tgt - op) * sigmoid_derivative(op)
            output_errors[i] = (targets[i] - self.final_outputs[i]) * self.sigmoid_derivative(self.final_outputs[i])
        
        # calculate hidden layer error
        hidden_error = [0] * self.hidden_size
        for i in range(self.hidden_size):
            error = 0
            # sum of error from each output neuron
            for j in range(self.output_size):
                error += output_errors[j] * self.weights_ho[j][i]
            hidden_error[i] = error *self.sigmoid_derivative(self.hidden_outputs[i])

        # Update weight between hidden and output layer
        for i in range(self.output_size):
            for j in range(self.hidden_size):
                #  weight_change = learning_rate * error * input
                self.weights_ho[i][j] += self.learning_rate * output_errors[i] * self.hidden_outputs[j]
            self.bias_output[i] += self.learning_rate * output_errors[i]

        for i in range(self.hidden_size):
            for j in range(self.input_size):
                self.weights_ih[i][j] += self.learning_rate * hidden_error[i] * self.inputs[j]
            self.bias_hidden[i] += self.learning_rate * hidden_error[i]
    
    def train(self, 
              training_data, targets, 
              epochs):
        for epoch in range(epochs):
            total_error = 0
            
            for i in range(len(training_data)):
                # forward pass
                outputs = self.forward(training_data[i])

                # mse
                error = sum([
                    (targets[i][j] - outputs[j]) ** 2 for j in range(len(outputs))
                ]) / len(outputs)
                total_error += error

                # backend pass
                self.backpropagate(targets[i])

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Error: {total_error / len(training_data)}")

    def predict(self, intputs):
        return self.forward(intputs) 

In [5]:
xor_inputs = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]

xor_outputs = [
    [0],
    [1],
    [1],
    [0]
]

nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1, learning_rate=0.3)
nn.train(xor_inputs, xor_outputs, 10000)

for i in range(len(xor_inputs)):
    prediction = nn.predict(xor_inputs[i])
    print(f"Input: {xor_inputs[i]}, Predicted: {prediction[0]:.4f}, Target: {xor_outputs[i][0]}")

Epoch 0, Error: 0.3105624439955551
Epoch 100, Error: 0.26158264759309935
Epoch 200, Error: 0.2582664561220185
Epoch 300, Error: 0.25500989070067054
Epoch 400, Error: 0.25220244990711266
Epoch 500, Error: 0.25000622489037067
Epoch 600, Error: 0.24841935763014403
Epoch 700, Error: 0.24734042808343817
Epoch 800, Error: 0.2466222808603685
Epoch 900, Error: 0.24613484485501477
Epoch 1000, Error: 0.24578807793772314
Epoch 1100, Error: 0.2455258756514423
Epoch 1200, Error: 0.24531453058862668
Epoch 1300, Error: 0.2451338658297102
Epoch 1400, Error: 0.24497165582846145
Epoch 1500, Error: 0.2448203522329112
Epoch 1600, Error: 0.24467520012539437
Epoch 1700, Error: 0.24453314933994744
Epoch 1800, Error: 0.24439221475863815
Epoch 1900, Error: 0.24425109209807921
Epoch 2000, Error: 0.2441089216614506
Epoch 2100, Error: 0.24396513968468733
Epoch 2200, Error: 0.2438193827568945
Epoch 2300, Error: 0.2436714251432573
Epoch 2400, Error: 0.24352113695146935
Epoch 2500, Error: 0.24336845576861524
Epoch 2